# Recoinize

[https://github.com/nicknochnack/FaceRecognition](https://github.com/nicknochnack/FaceRecognition/blob/main/Facial%20Verification%20with%20a%20Siamese%20Network%20-%20Final.ipynb)

# Setup

## Pull code from GitHub

In [2]:
%load_ext autoreload
%autoreload 2

from src.helpers.cmd import git_update, print_gpu_name
from src.helpers.install import install_requirements
# git_update("live-edit", force=False, show_output=False)
# install_requirements()

In [ ]:
!git fetch
!git pull

In [ ]:
print_gpu_name()

## Import dependencies

In [4]:
testing = False

In [66]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

# Import tensorflow dependencies - Functional API
if not testing:
    import tensorflow as tf
    from tensorflow import keras
    from keras.layers import Dense, Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
    from keras.models import Sequential, load_model, Model
    from datasets import load_dataset

    from src.models import make_embedding, make_siamese_model
    from src.helpers.setup import set_gpus_growth
    from src.dataset import create_pairs
    from src.preprocess import preprocess

    set_gpus_growth()
else:
    print("Testing mode, skipping tensorflow imports")
    from src.helpers.load_data import load_data
    from src.dataset import create_pairs


# Get and preprocess data

In [129]:
# Load the dataset from HuggingFace
if not testing:
    dataset = load_dataset('photonsquid/coins-euro')
    # convert the dataset to tensorflow dataset
    dataset = dataset.with_format('tf')
else:
    # load data from local files
    dataset = load_data("data/tests")

Using custom data configuration photonsquid--coins-euro-b75261052d3e19e0
Reusing dataset parquet (/root/.cache/huggingface/datasets/photonsquid___parquet/photonsquid--coins-euro-b75261052d3e19e0/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/3 [00:00<?, ?it/s]

In [135]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
# create the pairs and labels for the training dataset
train_dataset = create_p airs(train_dataset)

# convert it to a tensorflow dataset
# train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset)

In [ ]:
# show first pair in subplot with plt
# plt.subplot(1, 2, 1)
# plt.imshow(train_dataset[0][0]["bytes"])
# plt.axis('off')
# plt.title("anchor")
# plt.subplot(1, 2, 2)
# plt.imshow(train_dataset[0][1]["bytes"])
# plt.axis('off')
# plt.title("positive" if sum(train_dataset[0][2]) == 5 else "negative")
# plt.rcParams['figure.facecolor'] = '#282A36'
# plt.rcParams['text.color'] = '#ecf0f1'
# plt.show()

# decode the next image
# convert dataset into a numpy iterator
train_dataset = train_dataset.as_numpy_iterator()
# decode the next image
image = next(train_dataset)
# convert the bytes to a numpy array
image = np.frombuffer(image[0]["bytes"], dtype=np.uint8)
# plot it with matplotlib
# convert the bytes to a numpy array
image = image.reshape((256, 256, 3))
# plot it
plt.imshow(image)
plt.show()


# plot it with matplotlib
# convert the bytes to a numpy array

# image = image.reshape((256, 256, 3))
# plot it
# plt.imshow(image)
# plt.show()

In [ ]:
# prepare the dataset for training
train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.batch(32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

# prepare the dataset for testing
test_dataset = test_dataset.shuffle(1000)
test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)